# Text

In [1]:
## Dataset Order, you can also evaluate separately and use the corresponding code for evaluation

#### semeval2018_english_test,semeval2018_arabic_test,semeval2018_spanish_test,EWECT_usual_test,EWECT_virus_test,onlineshopping_test,MMS_test,XED_test

In [1]:
from utils.tools import *
import pandas as pd
import json
import csv

file_path = "./predicts/texts/generated_predictions.jsonl"
data = []

with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        data.append(json.loads(line.strip()))

df = pd.DataFrame(data)

df_semeval2018_english = df[:13269].reset_index(drop=True)
df_semeval2018_arabic = df[13269:19373].reset_index(drop=True)
df_semeval2018_spanish = df[19373:28755].reset_index(drop=True)
EWECT_usual_test = df[28755:33755].reset_index(drop=True)
EWECT_virus_test = df[33755:36755].reset_index(drop=True)
onlineshopping_test = df[36755:39255].reset_index(drop=True)
MMS_test = df[39255:52755].reset_index(drop=True)
XED_test = df[52755:65255].reset_index(drop=True)

df

,prompt,predict,label
0,You are a helpful assistant for affective anal...,The intensity score for Emotion E (anger) in t...,0.734
1,You are a helpful assistant for affective anal...,"Based on the given text, I would assess the em...",0.422
2,You are a helpful assistant for affective anal...,The intensity of emotion E in the text can be ...,0.663
3,You are a helpful assistant for affective anal...,"Based on the text provided, the emotion E, whi...",0.703
4,You are a helpful assistant for affective anal...,The intensity score for emotion E (anger) in t...,0.719
...,...,...,...
65250,You are a helpful assistant for affective anal...,{7}. Anticipation,2. sadness
65251,You are a helpful assistant for affective anal...,{4}. Fear,4. fear
65252,You are a helpful assistant for affective anal...,{1}. Disgust,2. sadness
65253,You are a helpful assistant for affective anal...,{1}. Disgust,2. sadness


In [2]:
results_write = []

In [3]:
# SemEval_en
from sklearn.metrics import f1_score, jaccard_score, accuracy_score

df_EIreg = df_semeval2018_english[:4068].reset_index(drop=True)
df_EIoc = df_semeval2018_english[4068:8136].reset_index(drop=True)
df_Voc = df_semeval2018_english[8136:9073].reset_index(drop=True)
df_Vreg = df_semeval2018_english[9073:10010].reset_index(drop=True)
df_Ec = df_semeval2018_english[10010:13269].reset_index(drop=True)

# EIreg
df_EIreg["predict_"] = df_EIreg["predict"].apply(get_scores_EIreg)
df_EIreg["label_"] = df_EIreg["label"].apply(get_scores_EIreg)
eva_EIreg_anger = df_EIreg[0:1002]
eva_EIreg_fear = df_EIreg[1002:1988].reset_index(drop=True)
eva_EIreg_joy = df_EIreg[1988:3093].reset_index(drop=True)
eva_EIreg_sadness = df_EIreg[3093:4068].reset_index(drop=True)
anger_correlation = eva_EIreg_anger['predict_'].corr(eva_EIreg_anger['label_'], method='pearson')
fear_correlation = eva_EIreg_fear['predict_'].corr(eva_EIreg_fear['label_'], method='pearson')
joy_correlation = eva_EIreg_joy['predict_'].corr(eva_EIreg_joy['label_'], method='pearson')
sadness_correlation = eva_EIreg_sadness['predict_'].corr(eva_EIreg_sadness['label_'], method='pearson')
ave = np.mean([anger_correlation,fear_correlation,joy_correlation,sadness_correlation])
print("EIreg: ","ave","anger","fear","joy","sadness")
print(ave,anger_correlation,fear_correlation,joy_correlation,sadness_correlation)
results_write.extend([ave,anger_correlation,fear_correlation,joy_correlation,sadness_correlation])

# EIoc
df_EIoc["predict_"] = df_EIoc["predict"].apply(extract_integers_EIoc)
df_EIoc["label_"] = df_EIoc["label"].apply(extract_integers_EIoc)
eva_EIoc_anger = df_EIoc[0:1002]
eva_EIoc_fear = df_EIoc[1002:1988].reset_index(drop=True)
eva_EIoc_joy = df_EIoc[1988:3093].reset_index(drop=True)
eva_EIoc_sadness = df_EIoc[3093:4068].reset_index(drop=True)
anger_correlation = eva_EIoc_anger['predict_'].corr(eva_EIoc_anger['label_'], method='pearson')
fear_correlation = eva_EIoc_fear['predict_'].corr(eva_EIoc_fear['label_'], method='pearson')
joy_correlation = eva_EIoc_joy['predict_'].corr(eva_EIoc_joy['label_'], method='pearson')
sadness_correlation = eva_EIoc_sadness['predict_'].corr(eva_EIoc_sadness['label_'], method='pearson')
ave = np.mean([anger_correlation,fear_correlation,joy_correlation,sadness_correlation])
print("EIreg: ","ave","anger","fear","joy","sadness")
print(ave,anger_correlation,fear_correlation,joy_correlation,sadness_correlation)
results_write.extend([ave,anger_correlation,fear_correlation,joy_correlation,sadness_correlation])


# Voc
df_Voc["predict_"] = df_Voc["predict"].apply(extract_integers_Voc)
df_Voc["label_"] = df_Voc["label"].apply(extract_integers_Voc)
correlation = df_Voc['predict_'].corr(df_Voc['label_'], method='pearson')
print("Voc")
print(correlation)
results_write.extend([correlation])

# Vreg
df_Vreg["predict_"] = df_Vreg["predict"].apply(get_scores_Vreg)
df_Vreg["label_"] = df_Vreg["label"].apply(get_scores_Vreg)
correlation = df_Vreg['predict_'].corr(df_Vreg['label_'], method='pearson')
print("Vreg")
print(correlation)
results_write.extend([correlation])


# Ec
df_Ec["label_"] = df_Ec["label"].apply(parse_labels_Ec)
df_Ec["predict_"] = df_Ec["predict"].apply(parse_labels_Ec)
y_true = np.array(df_Ec["label_"].tolist())
y_pred = np.array(df_Ec["predict_"].tolist())
micro_f1 = f1_score(y_true, y_pred, average='micro')
macro_f1 = f1_score(y_true, y_pred, average='macro', zero_division=1)  
# accuracy = accuracy_score(y_true, y_pred)
accuracy = jaccard_score(y_true,y_pred, average='weighted')
print("Ec:","acc","mic-f1","mac-f1")
print(accuracy,micro_f1,macro_f1)
results_write.extend([accuracy,micro_f1,macro_f1])


EIreg:  ave anger fear joy sadness
0.0863936187983066 0.1889495569491233 0.03719642639432761 0.09427879294351836 0.025149698906257185
EIreg:  ave anger fear joy sadness
0.11368090574128496 0.09179560760422588 0.10615389545331783 0.059704810967509986 0.19706930894008617
Voc
0.35337451559813143
Vreg
0.6187461292007865
Ec: acc mic-f1 mac-f1
0.18851806065659305 0.31202907543657477 0.24646315715136993


In [4]:
# SemEval_ar

df_EIreg = df_semeval2018_arabic[:1563].reset_index(drop=True)
df_EIoc = df_semeval2018_arabic[1563:3126].reset_index(drop=True)
df_Voc = df_semeval2018_arabic[3126:3856].reset_index(drop=True)
df_Vreg = df_semeval2018_arabic[3856:4586].reset_index(drop=True)
df_Ec = df_semeval2018_arabic[4586:].reset_index(drop=True)

# EIreg
df_EIreg["predict_"] = df_EIreg["predict"].apply(get_scores_EIreg)
df_EIreg["label_"] = df_EIreg["label"].apply(get_scores_EIreg)
eva_EIreg_anger = df_EIreg[0:373]
eva_EIreg_fear = df_EIreg[373:745].reset_index(drop=True)
eva_EIreg_joy = df_EIreg[745:1193].reset_index(drop=True)
eva_EIreg_sadness = df_EIreg[1193:1563].reset_index(drop=True)
anger_correlation = eva_EIreg_anger['predict_'].corr(eva_EIreg_anger['label_'], method='pearson')
fear_correlation = eva_EIreg_fear['predict_'].corr(eva_EIreg_fear['label_'], method='pearson')
joy_correlation = eva_EIreg_joy['predict_'].corr(eva_EIreg_joy['label_'], method='pearson')
sadness_correlation = eva_EIreg_sadness['predict_'].corr(eva_EIreg_sadness['label_'], method='pearson')
ave = np.mean([anger_correlation,fear_correlation,joy_correlation,sadness_correlation])
print("EIreg: ","ave","anger","fear","joy","sadness")
print(ave,anger_correlation,fear_correlation,joy_correlation,sadness_correlation)
results_write.extend([ave,anger_correlation,fear_correlation,joy_correlation,sadness_correlation])

# EIoc
df_EIoc["predict_"] = df_EIoc["predict"].apply(extract_integers_EIoc)
df_EIoc["label_"] = df_EIoc["label"].apply(extract_integers_EIoc)
eva_EIoc_anger = df_EIoc[0:373]
eva_EIoc_fear = df_EIoc[373:745].reset_index(drop=True)
eva_EIoc_joy = df_EIoc[745:1193].reset_index(drop=True)
eva_EIoc_sadness = df_EIoc[1193:1563].reset_index(drop=True)
anger_correlation = eva_EIoc_anger['predict_'].corr(eva_EIoc_anger['label_'], method='pearson')
fear_correlation = eva_EIoc_fear['predict_'].corr(eva_EIoc_fear['label_'], method='pearson')
joy_correlation = eva_EIoc_joy['predict_'].corr(eva_EIoc_joy['label_'], method='pearson')
sadness_correlation = eva_EIoc_sadness['predict_'].corr(eva_EIoc_sadness['label_'], method='pearson')
ave = np.mean([anger_correlation,fear_correlation,joy_correlation,sadness_correlation])
print("EIreg: ","ave","anger","fear","joy","sadness")
print(ave,anger_correlation,fear_correlation,joy_correlation,sadness_correlation)
results_write.extend([ave,anger_correlation,fear_correlation,joy_correlation,sadness_correlation])

# Voc
df_Voc["predict_"] = df_Voc["predict"].apply(extract_integers_Voc)
df_Voc["label_"] = df_Voc["label"].apply(extract_integers_Voc)
correlation = df_Voc['predict_'].corr(df_Voc['label_'], method='pearson')
print("Voc")
print(correlation)
results_write.extend([correlation])

# Vreg
df_Vreg["predict_"] = df_Vreg["predict"].apply(get_scores_Vreg)
df_Vreg["label_"] = df_Vreg["label"].apply(get_scores_Vreg)
correlation = df_Vreg['predict_'].corr(df_Vreg['label_'], method='pearson')
print("Vreg")
print(correlation)
results_write.extend([correlation])

# Ec
df_Ec["label_"] = df_Ec["label"].apply(parse_labels_Ec)
df_Ec["predict_"] = df_Ec["predict"].apply(parse_labels_Ec)
y_true = np.array(df_Ec["label_"].tolist())
y_pred = np.array(df_Ec["predict_"].tolist())
micro_f1 = f1_score(y_true, y_pred, average='micro')
macro_f1 = f1_score(y_true, y_pred, average='macro', zero_division=1) 
# accuracy = accuracy_score(y_true, y_pred)
accuracy = jaccard_score(y_true,y_pred, average='weighted')
print("Ec:","acc","mic-f1","mac-f1")
print(accuracy,micro_f1,macro_f1)
results_write.extend([accuracy,micro_f1,macro_f1])


EIreg:  ave anger fear joy sadness
-0.02431572020536795 -0.03739759246690567 -0.09766690700886374 0.06715358144805039 -0.029351962793752788
EIreg:  ave anger fear joy sadness
0.022674823541623736 0.06000329487754422 0.02966064469170994 -0.01324945896824079 0.014284813565481573
Voc
0.24788638378784178
Vreg
0.3534154041704705
Ec: acc mic-f1 mac-f1
0.11199856651279064 0.20344894400310015 0.14388365783642576


In [5]:
# SemEval_spanish

df_EIreg = df_semeval2018_spanish[:2616].reset_index(drop=True)
df_EIoc = df_semeval2018_spanish[2616:5232].reset_index(drop=True)
df_Voc = df_semeval2018_spanish[5232:5880].reset_index(drop=True)
df_Vreg = df_semeval2018_spanish[5880:6528].reset_index(drop=True)
df_Ec = df_semeval2018_spanish[6528:9382].reset_index(drop=True)

# EIreg
df_EIreg["predict_"] = df_EIreg["predict"].apply(get_scores_EIreg)
df_EIreg["label_"] = df_EIreg["label"].apply(get_scores_EIreg)
eva_EIreg_anger = df_EIreg[0:627]
eva_EIreg_fear = df_EIreg[627:1245].reset_index(drop=True)
eva_EIreg_joy = df_EIreg[1245:1975].reset_index(drop=True)
eva_EIreg_sadness = df_EIreg[1975:2616].reset_index(drop=True)
anger_correlation = eva_EIreg_anger['predict_'].corr(eva_EIreg_anger['label_'], method='pearson')
fear_correlation = eva_EIreg_fear['predict_'].corr(eva_EIreg_fear['label_'], method='pearson')
joy_correlation = eva_EIreg_joy['predict_'].corr(eva_EIreg_joy['label_'], method='pearson')
sadness_correlation = eva_EIreg_sadness['predict_'].corr(eva_EIreg_sadness['label_'], method='pearson')
ave = np.mean([anger_correlation,fear_correlation,joy_correlation,sadness_correlation])
print("EIreg: ","ave","anger","fear","joy","sadness")
print(ave,anger_correlation,fear_correlation,joy_correlation,sadness_correlation)
results_write.extend([ave,anger_correlation,fear_correlation,joy_correlation,sadness_correlation])

# EIoc
df_EIoc["predict_"] = df_EIoc["predict"].apply(extract_integers_EIoc)
df_EIoc["label_"] = df_EIoc["label"].apply(extract_integers_EIoc)
eva_EIoc_anger = df_EIoc[0:627]
eva_EIoc_fear = df_EIoc[627:1245].reset_index(drop=True)
eva_EIoc_joy = df_EIoc[1245:1975].reset_index(drop=True)
eva_EIoc_sadness = df_EIoc[1975:2616].reset_index(drop=True)
anger_correlation = eva_EIoc_anger['predict_'].corr(eva_EIoc_anger['label_'], method='pearson')
fear_correlation = eva_EIoc_fear['predict_'].corr(eva_EIoc_fear['label_'], method='pearson')
joy_correlation = eva_EIoc_joy['predict_'].corr(eva_EIoc_joy['label_'], method='pearson')
sadness_correlation = eva_EIoc_sadness['predict_'].corr(eva_EIoc_sadness['label_'], method='pearson')
ave = np.mean([anger_correlation,fear_correlation,joy_correlation,sadness_correlation])
print("EIreg: ","ave","anger","fear","joy","sadness")
print(ave,anger_correlation,fear_correlation,joy_correlation,sadness_correlation)
results_write.extend([ave,anger_correlation,fear_correlation,joy_correlation,sadness_correlation])

# Voc
df_Voc["predict_"] = df_Voc["predict"].apply(extract_integers_Voc)
df_Voc["label_"] = df_Voc["label"].apply(extract_integers_Voc)
correlation = df_Voc['predict_'].corr(df_Voc['label_'], method='pearson')
print("Voc")
print(correlation)
results_write.extend([correlation])

# Vreg
df_Vreg["predict_"] = df_Vreg["predict"].apply(get_scores_Vreg)
df_Vreg["label_"] = df_Vreg["label"].apply(get_scores_Vreg)
correlation = df_Vreg['predict_'].corr(df_Vreg['label_'], method='pearson')
print("Vreg")
print(correlation)
results_write.extend([correlation])

# Ec
df_Ec["label_"] = df_Ec["label"].apply(parse_labels_Ec)
df_Ec["predict_"] = df_Ec["predict"].apply(parse_labels_Ec)
y_true = np.array(df_Ec["label_"].tolist())
y_pred = np.array(df_Ec["predict_"].tolist())
micro_f1 = f1_score(y_true, y_pred, average='micro')
macro_f1 = f1_score(y_true, y_pred, average='macro', zero_division=1)
# accuracy = accuracy_score(y_true, y_pred)
accuracy = jaccard_score(y_true,y_pred, average='weighted')
print("Ec:","acc","mic-f1","mac-f1")
print(accuracy,micro_f1,macro_f1)
results_write.extend([accuracy,micro_f1,macro_f1])

EIreg:  ave anger fear joy sadness
0.029362212595999207 0.12142366538495562 -0.05525225530952126 0.12410116977652466 -0.0728237294679622
EIreg:  ave anger fear joy sadness
0.06666450614385279 0.06498573964489061 0.04900829362119919 0.040406528866702554 0.11225746244261882
Voc
0.2963341507639899
Vreg
0.4212872911736552
Ec: acc mic-f1 mac-f1
0.20092512644182206 0.328622816032888 0.23759532351863866


In [6]:
EWECT_usual_test["predict_"] = EWECT_usual_test["predict"].apply(parse_labels_EWECT)
EWECT_usual_test["label_"] = EWECT_usual_test["label"].apply(parse_labels_EWECT)
y_true = np.array(EWECT_usual_test["label_"].tolist())
y_pred = np.array(EWECT_usual_test["predict_"].tolist())
micro_f1 = f1_score(y_true, y_pred, average='micro')
macro_f1 = f1_score(y_true, y_pred, average='macro', zero_division=1)  
accuracy = accuracy_score(y_true, y_pred)
print("EWECT_usual:","acc", "mic-f1","mac-f1")
print(accuracy,micro_f1,macro_f1)
results_write.extend([accuracy,macro_f1])

EWECT_virus_test["predict_"] = EWECT_virus_test["predict"].apply(parse_labels_EWECT)
EWECT_virus_test["label_"] = EWECT_virus_test["label"].apply(parse_labels_EWECT)
y_true = np.array(EWECT_virus_test["label_"].tolist())
y_pred = np.array(EWECT_virus_test["predict_"].tolist())
micro_f1 = f1_score(y_true, y_pred, average='micro')
macro_f1 = f1_score(y_true, y_pred, average='macro', zero_division=1)  
accuracy = accuracy_score(y_true, y_pred)
print("EWECT_virus:","acc", "mic-f1","mac-f1")
print(accuracy,micro_f1,macro_f1)
results_write.extend([accuracy,macro_f1])


EWECT_usual: acc mic-f1 mac-f1
0.4262 0.43430017519953285 0.37747380139331965
EWECT_virus: acc mic-f1 mac-f1
0.4053333333333333 0.4108614837562632 0.298011970376541


In [7]:
onlineshopping_test["predict_"] = onlineshopping_test["predict"].apply(extract_label_onlineshopping)
onlineshopping_test["label_"] = onlineshopping_test["label"].apply(extract_label_onlineshopping)
accuracy = accuracy_score(onlineshopping_test['label_'], onlineshopping_test['predict_'])
macro_f1 = f1_score(onlineshopping_test['label_'], onlineshopping_test['predict_'], average='macro')
print("onlineshopping:","acc", "mac-f1")
print(accuracy,macro_f1)
results_write.extend([accuracy,macro_f1])

onlineshopping: acc mac-f1
0.8316 0.5713836082512772


In [8]:
MMS_test["predict_"] = MMS_test["predict"].apply(extract_label_MMS)
MMS_test["label_"] = MMS_test["label"].apply(extract_label_MMS)
accuracy = accuracy_score(MMS_test['label_'], MMS_test['predict_'])
macro_f1 = f1_score(MMS_test['label_'], MMS_test['predict_'], average='macro')
print("MMS:","acc", "mac-f1")
print(accuracy,macro_f1)
results_write.extend([accuracy,macro_f1])

MMS: acc mac-f1
0.4617777777777778 0.3170070257648976


In [9]:
results = []

num_languages = 27
rows_per_language = 500

for i in range(num_languages):
    start_idx = i * rows_per_language
    end_idx = start_idx + rows_per_language
    subset = MMS_test.iloc[start_idx:end_idx]
    
    accuracy = accuracy_score(subset['label_'], subset['predict_'])
    macro_f1 = f1_score(subset['label_'], subset['predict_'], average='macro')
    
    results.append({'language': f'lang_{i+1}', 'accuracy': accuracy, 'macro_f1': macro_f1})

df_results = pd.DataFrame(results)
df_results.to_csv("./results/MMS_languages.csv",index=False)
print(df_results)

   language  accuracy  macro_f1
0    lang_1     0.376  0.271393
1    lang_2     0.430  0.297476
2    lang_3     0.452  0.292904
3    lang_4     0.600  0.382165
4    lang_5     0.486  0.440305
5    lang_6     0.690  0.375553
6    lang_7     0.540  0.352516
7    lang_8     0.396  0.293230
8    lang_9     0.516  0.452381
9   lang_10     0.414  0.253825
10  lang_11     0.472  0.318554
11  lang_12     0.562  0.345348
12  lang_13     0.558  0.429235
13  lang_14     0.432  0.281692
14  lang_15     0.490  0.362297
15  lang_16     0.370  0.338617
16  lang_17     0.472  0.408281
17  lang_18     0.378  0.345012
18  lang_19     0.472  0.346551
19  lang_20     0.562  0.327781
20  lang_21     0.386  0.272743
21  lang_22     0.470  0.260986
22  lang_23     0.322  0.224737
23  lang_24     0.436  0.310100
24  lang_25     0.276  0.228621
25  lang_26     0.378  0.276563
26  lang_27     0.532  0.385251


In [11]:
# df["prompt"][0]

In [12]:
XED_test["predict_"] = XED_test["predict"].apply(parse_labels_XED)
XED_test["label_"] = XED_test["label"].apply(parse_labels_XED)
y_true = np.array(XED_test["label_"].tolist())
y_pred = np.array(XED_test["predict_"].tolist())
micro_f1 = f1_score(y_true, y_pred, average='micro')
macro_f1 = f1_score(y_true, y_pred, average='macro', zero_division=1)  
# accuracy = accuracy_score(y_true, y_pred)
accuracy = jaccard_score(y_true,y_pred, average='weighted')
print("XED:","acc", "mic-f1","mac-f1")
print(accuracy,micro_f1,macro_f1)
results_write.extend([accuracy,micro_f1,macro_f1])


XED: acc mic-f1 mac-f1
0.1271803411429323 0.21537853553886974 0.19034426575335456


In [13]:
num_languages = 25
rows_per_language = 500
results = []
for i in range(num_languages):
    start_idx = i * rows_per_language
    end_idx = start_idx + rows_per_language
    subset = XED_test.iloc[start_idx:end_idx]
    
    y_true = np.array(subset["label_"].tolist())
    y_pred = np.array(subset["predict_"].tolist())
    
#     accuracy = accuracy_score(y_true, y_pred)
    accuracy = jaccard_score(y_true,y_pred, average='weighted')
    micro_f1 = f1_score(y_true, y_pred, average='micro')
    macro_f1 = f1_score(y_true, y_pred, average='macro', zero_division=1)
    
    results.append({
        'language': f'lang_{i+1}', 
        'accuracy': accuracy, 
        'macro_f1': macro_f1,
        'micro_f1': micro_f1 
        
    })
#     break

df_results = pd.DataFrame(results)
df_results.to_csv("./results/XED_languages.csv",index=False)
print("XED: acc, mic-f1, mac-f1")
print(df_results)

XED: acc, mic-f1, mac-f1
   language  accuracy  macro_f1  micro_f1
0    lang_1  0.145462  0.201600  0.227155
1    lang_2  0.112672  0.176896  0.193122
2    lang_3  0.127127  0.187975  0.222387
3    lang_4  0.115113  0.174191  0.199228
4    lang_5  0.131230  0.187952  0.219986
5    lang_6  0.132409  0.193529  0.230548
6    lang_7  0.121708  0.187838  0.208254
7    lang_8  0.207228  0.282119  0.327994
8    lang_9  0.150830  0.214325  0.253763
9   lang_10  0.103087  0.162577  0.176923
10  lang_11  0.142164  0.199003  0.242245
11  lang_12  0.108017  0.169126  0.189020
12  lang_13  0.119370  0.190382  0.208426
13  lang_14  0.119214  0.178224  0.207042
14  lang_15  0.140838  0.206997  0.237637
15  lang_16  0.121237  0.177542  0.202279
16  lang_17  0.116830  0.171325  0.190803
17  lang_18  0.110943  0.166386  0.187320
18  lang_19  0.151923  0.219991  0.256677
19  lang_20  0.116916  0.176982  0.199728
20  lang_21  0.138022  0.205870  0.238889
21  lang_22  0.119501  0.174044  0.199723
22  lang_

In [14]:
# change to DataFrame
df_results_write = pd.DataFrame([results_write])

# write to CSV fule
df_results_write.to_csv("./results/results_text.csv", index=False, header=False, encoding="utf-8")

# Image

In [ ]:
## Dataset Order, you can also evaluate separately and use the corresponding code for evaluation
#### EMOTIC_EC_test,EMOTIC_SI_test,FER2013_test,CFAPS_EC_test,CFAPS_EI_test

In [1]:
from utils.tools import *
import pandas as pd
import json

file_path = "./predicts/images/generated_predictions.jsonl"
data = []

with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        data.append(json.loads(line.strip()))
        
df = pd.DataFrame(data)

EMOTIC_seg_emo_test = df[:7203].reset_index(drop=True)
EMOTIC_seg_sentiment_test = df[7203:14406].reset_index(drop=True)
FER2013_test = df[14406:21584].reset_index(drop=True)
CFAPS_emo_test = df[21584:21758].reset_index(drop=True)
CFAPS_emoint_test = df[21758:21932].reset_index(drop=True)

df

,prompt,predict,label
0,<|im_start|>system\nYou are a helpful assistan...,18. engagement,"1. happiness, 8. peace, 9. affection, 12. conf..."
1,<|im_start|>system\nYou are a helpful assistan...,"1. happiness, 18. engagement","1. happiness, 7. excitement, 8. peace, 13. dis..."
2,<|im_start|>system\nYou are a helpful assistan...,1. happiness,"7. excitement, 11. anticipation, 14. disconnec..."
3,<|im_start|>system\nYou are a helpful assistan...,"1. happiness, 22. pleasure","1. happiness, 8. peace, 12. confidence, 22. pl..."
4,<|im_start|>system\nYou are a helpful assistan...,1. happiness,"1. happiness, 7. excitement, 8. peace, 11. ant..."
...,...,...,...
21927,<|im_start|>system\nYou are a helpful assistan...,0.707894736842105,0.7604166666666663\n
21928,<|im_start|>system\nYou are a helpful assistan...,0.59375,0.5548780487804875\n
21929,<|im_start|>system\nYou are a helpful assistan...,0.630434782608695,0.5885416666666663\n
21930,<|im_start|>system\nYou are a helpful assistan...,0.578125,0.601190476190475\n


In [2]:
CFAPS_emoint_test

,prompt,predict,label
0,<|im_start|>system\nYou are a helpful assistan...,0.5578125,0.460843373493975\n
1,<|im_start|>system\nYou are a helpful assistan...,0.677195945945945,0.7927215189873412\n
2,<|im_start|>system\nYou are a helpful assistan...,0.60546875,0.6021341463414625\n
3,<|im_start|>system\nYou are a helpful assistan...,0.6470588235294113,0.7938311688311688\n
4,<|im_start|>system\nYou are a helpful assistan...,0.46774193548387,0.5460526315789462\n
...,...,...,...
169,<|im_start|>system\nYou are a helpful assistan...,0.707894736842105,0.7604166666666663\n
170,<|im_start|>system\nYou are a helpful assistan...,0.59375,0.5548780487804875\n
171,<|im_start|>system\nYou are a helpful assistan...,0.630434782608695,0.5885416666666663\n
172,<|im_start|>system\nYou are a helpful assistan...,0.578125,0.601190476190475\n


In [3]:
results_write = []

In [4]:
EMOTIC_seg_emo_test["predict_"] = EMOTIC_seg_emo_test["predict"].apply(parse_labels_EMOTIC_emo)
EMOTIC_seg_emo_test["label_"] = EMOTIC_seg_emo_test["label"].apply(parse_labels_EMOTIC_emo)
y_true = np.array(EMOTIC_seg_emo_test["label_"].tolist())
y_pred = np.array(EMOTIC_seg_emo_test["predict_"].tolist())
micro_f1 = f1_score(y_true, y_pred, average='micro')
macro_f1 = f1_score(y_true, y_pred, average='macro', zero_division=1)  
# accuracy = accuracy_score(y_true, y_pred)
accuracy = jaccard_score(y_true,y_pred, average='weighted')
print("EMOTIC_emo:","acc", "mic-f1", "mac-f1")
print(accuracy,micro_f1,macro_f1)
results_write.extend([accuracy,micro_f1,macro_f1])

EMOTIC_seg_sentiment_test[['predict_valence', 'predict_arousal', 'predict_dominance']] = EMOTIC_seg_sentiment_test['predict'].apply(lambda x: pd.Series(extract_values_EMOTIC_vad(x)))
EMOTIC_seg_sentiment_test[['label_valence', 'label_arousal', 'label_dominance']] = EMOTIC_seg_sentiment_test['label'].apply(lambda x: pd.Series(extract_values_EMOTIC_vad(x)))

valence_corr = calculate_pearson_correlation(EMOTIC_seg_sentiment_test['predict_valence'], EMOTIC_seg_sentiment_test['label_valence'])
arousal_corr = calculate_pearson_correlation(EMOTIC_seg_sentiment_test['predict_arousal'], EMOTIC_seg_sentiment_test['label_arousal'])
dominance_corr = calculate_pearson_correlation(EMOTIC_seg_sentiment_test['predict_dominance'], EMOTIC_seg_sentiment_test['label_dominance'])

print("EMOTIC_vad:","V","A","D")
print(valence_corr,arousal_corr,dominance_corr)
results_write.extend([valence_corr,arousal_corr,dominance_corr])

FER2013_test["predict_"] = FER2013_test["predict"].apply(parse_labels_FER2013)
FER2013_test["label_"] = FER2013_test["label"].apply(parse_labels_FER2013)
y_true = np.array(FER2013_test["label_"].tolist())
y_pred = np.array(FER2013_test["predict_"].tolist())
micro_f1 = f1_score(y_true, y_pred, average='micro')
macro_f1 = f1_score(y_true, y_pred, average='macro', zero_division=1)  
accuracy = accuracy_score(y_true, y_pred)
print("FER2013:","acc", "mic-f1", "mac-f1")
print(accuracy,micro_f1,macro_f1)
results_write.extend([accuracy,macro_f1])

CFAPS_emo_test["predict_"] = CFAPS_emo_test["predict"].apply(parse_labels_CFAPS_emo)
CFAPS_emo_test["label_"] = CFAPS_emo_test["label"].apply(parse_labels_CFAPS_emo)
y_true = np.array(CFAPS_emo_test["label_"].tolist())
y_pred = np.array(CFAPS_emo_test["predict_"].tolist())
micro_f1 = f1_score(y_true, y_pred, average='micro')
macro_f1 = f1_score(y_true, y_pred, average='macro', zero_division=1)  
accuracy = accuracy_score(y_true, y_pred)
print("CFAPS_emo:","acc", "mic-f1", "mac-f1")
print(accuracy,micro_f1,macro_f1)
results_write.extend([accuracy,macro_f1])


CFAPS_emoint_test["predict_"] = CFAPS_emoint_test["predict"].apply(get_scores_CFAPS_emoint)
CFAPS_emoint_test["label_"] = CFAPS_emoint_test["label"].apply(get_scores_CFAPS_emoint)

correlation = CFAPS_emoint_test['predict_'].corr(CFAPS_emoint_test['label_'], method='pearson')
print("CFAPS_emoint")
print(correlation)
results_write.extend([correlation])



/Users/lzw/packages/anaconda3/envs/ai/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 in labels with no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


EMOTIC_emo: acc mic-f1 mac-f1
0.2269986218933977 0.38002218685187866 0.18972664472831802
EMOTIC_vad: V A D
0.47801115958052764 0.5355533270486557 0.2356522709128337
FER2013: acc mic-f1 mac-f1
0.553496795764837 0.553496795764837 0.4595078967261605
CFAPS_emo: acc mic-f1 mac-f1
0.8908045977011494 0.8908045977011495 0.8281739515166481
CFAPS_emoint
0.8430492204522055


In [5]:
CFAPS_emoint_test

,prompt,predict,label,predict_,label_
0,<|im_start|>system\nYou are a helpful assistan...,0.5578125,0.460843373493975\n,0.557813,0.460843
1,<|im_start|>system\nYou are a helpful assistan...,0.677195945945945,0.7927215189873412\n,0.677196,0.792722
2,<|im_start|>system\nYou are a helpful assistan...,0.60546875,0.6021341463414625\n,0.605469,0.602134
3,<|im_start|>system\nYou are a helpful assistan...,0.6470588235294113,0.7938311688311688\n,0.647059,0.793831
4,<|im_start|>system\nYou are a helpful assistan...,0.46774193548387,0.5460526315789462\n,0.467742,0.546053
...,...,...,...,...,...
169,<|im_start|>system\nYou are a helpful assistan...,0.707894736842105,0.7604166666666663\n,0.707895,0.760417
170,<|im_start|>system\nYou are a helpful assistan...,0.59375,0.5548780487804875\n,0.593750,0.554878
171,<|im_start|>system\nYou are a helpful assistan...,0.630434782608695,0.5885416666666663\n,0.630435,0.588542
172,<|im_start|>system\nYou are a helpful assistan...,0.578125,0.601190476190475\n,0.578125,0.601190


In [6]:
df_results_write = pd.DataFrame([results_write])
df_results_write.to_csv("./results/results_image.csv", index=False, header=False, encoding="utf-8")

# video

In [ ]:
## Dataset Order, you can also evaluate separately and use the corresponding code for evaluation

#### SAMSEMO_test,MELD_EC_test,MELD_SP_test,CHSIMS_SP_test,CHSIMS_SI_test

In [7]:
from utils.tools import *
import pandas as pd
import json

file_path = "./predicts/videos/generated_predictions.jsonl" 
data = []

with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        data.append(json.loads(line.strip()))
df = pd.DataFrame(data)

SAMSEMO_test = df[:2181].reset_index(drop=True)
MELD_emo_test = df[2181:4711].reset_index(drop=True)
MELD_Sentiment_test = df[4711:7241].reset_index(drop=True)
CHSIMS_polarity_test = df[7241:7698].reset_index(drop=True)
CHSIMS_strength_test = df[7698:8155].reset_index(drop=True)

df

,prompt,predict,label
0,<|im_start|>system\nYou are a helpful assistan...,0. neutral,5. surprise\n
1,<|im_start|>system\nYou are a helpful assistan...,0. neutral,0. neutral\n
2,<|im_start|>system\nYou are a helpful assistan...,0. neutral,0. neutral\n
3,<|im_start|>system\nYou are a helpful assistan...,1. happiness,1. happiness\n
4,<|im_start|>system\nYou are a helpful assistan...,1. happiness,1. happiness\n
...,...,...,...
8150,<|im_start|>system\nYou are a helpful assistan...,-0.4,-1.0\n
8151,<|im_start|>system\nYou are a helpful assistan...,-0.8,-0.2\n
8152,<|im_start|>system\nYou are a helpful assistan...,-1.0,-1.0\n
8153,<|im_start|>system\nYou are a helpful assistan...,-1.0,-1.0\n


In [8]:
results_write = []

In [9]:
import pandas as pd
import re
import string

SAMSEMO_test["predict_"] = SAMSEMO_test["predict"].apply(parse_labels_SAMSEMO)
SAMSEMO_test["label_"] = SAMSEMO_test["label"].apply(parse_labels_SAMSEMO)

def rmpunctuation(text):
    return ''.join(char for char in text if char not in string.punctuation)

def extract_content(text):
    clean_text = re.sub(r'<\|.*?\|>', '', text) 
    match = re.search(r'Content: (.*)', clean_text)
    if match:
        return match.group(1)  
    else:
        return None
    
def extract_content(text):
    clean_text = re.sub(r'<\|.*?\|>', '', text)  
    match = re.search(r'Content: (.*)', clean_text)
    if match:
        return match.group(1).replace(" ASSISTANT:","")  
    else:
        return None
    
SAMSEMO_original = pd.read_csv("./video/Sansemo/test.csv")
SAMSEMO_test["text"] = SAMSEMO_test["prompt"].apply(extract_content)
SAMSEMO_test["text"] = SAMSEMO_test["text"].apply(rmpunctuation)
merage_text = SAMSEMO_original[["transcript","labels","language"]]
merage_text["transcript"] = merage_text["transcript"].apply(rmpunctuation)
merged = pd.merge(SAMSEMO_test, merage_text, left_on='text', right_on='transcript', how='left')

merged["predict_"] = merged["predict"].apply(parse_labels_SAMSEMO)
merged["label_"] = merged["label"].apply(parse_labels_SAMSEMO)


SAMSEMO_DE = merged[merged["language"]=="DE"]
SAMSEMO_EN = merged[merged["language"]=="EN"]
SAMSEMO_ES = merged[merged["language"]=="ES"]
SAMSEMO_KO = merged[merged["language"]=="KO"]
SAMSEMO_PL = merged[merged["language"]=="PL"]

y_true = np.array(merged["label_"].tolist())
y_pred = np.array(merged["predict_"].tolist())
accuracy,micro_f1,macro_f1 = get_ACCandF1(y_true, y_pred)
print("SAMSEMO_macro", accuracy,micro_f1,macro_f1)
results_write.extend([accuracy,micro_f1,macro_f1])

y_true = np.array(SAMSEMO_DE["label_"].tolist())
y_pred = np.array(SAMSEMO_DE["predict_"].tolist())
accuracy,micro_f1,macro_f1 = get_ACCandF1(y_true, y_pred)
print("SAMSEMO_DE", accuracy,micro_f1,macro_f1)
results_write.extend([accuracy,micro_f1,macro_f1])

y_true = np.array(SAMSEMO_EN["label_"].tolist())
y_pred = np.array(SAMSEMO_EN["predict_"].tolist())
accuracy,micro_f1,macro_f1 = get_ACCandF1(y_true, y_pred)
print("SAMSEMO_EN", accuracy,micro_f1,macro_f1)
results_write.extend([accuracy,micro_f1,macro_f1])

y_true = np.array(SAMSEMO_ES["label_"].tolist())
y_pred = np.array(SAMSEMO_ES["predict_"].tolist())
accuracy,micro_f1,macro_f1 = get_ACCandF1(y_true, y_pred)
print("SAMSEMO_ES", accuracy,micro_f1,macro_f1)
results_write.extend([accuracy,micro_f1,macro_f1])

y_true = np.array(SAMSEMO_KO["label_"].tolist())
y_pred = np.array(SAMSEMO_KO["predict_"].tolist())
accuracy,micro_f1,macro_f1 = get_ACCandF1(y_true, y_pred)
print("SAMSEMO_KO", accuracy,micro_f1,macro_f1)
results_write.extend([accuracy,micro_f1,macro_f1])

y_true = np.array(SAMSEMO_PL["label_"].tolist())
y_pred = np.array(SAMSEMO_PL["predict_"].tolist())
accuracy,micro_f1,macro_f1 = get_ACCandF1(y_true, y_pred)
print("SAMSEMO_PL", accuracy,micro_f1,macro_f1)
results_write.extend([accuracy,micro_f1,macro_f1])


SAMSEMO_macro 0.513624592151392 0.674937965260546 0.5124364143850584
SAMSEMO_DE 0.43995908405167017 0.6116373477672531 0.5109276449746007
SAMSEMO_EN 0.4927058569278302 0.6627318718381112 0.6070522620884977
SAMSEMO_ES 0.49999269151212616 0.669051878354204 0.44977053915275994
SAMSEMO_KO 0.5235368059539747 0.6719999999999999 0.5732867415070806
SAMSEMO_PL 0.5790291450731813 0.7163029525032093 0.4309102670898862


/var/folders/xb/mtrqllcj6wv_x2vxx75t3t400000gn/T/ipykernel_19662/4058833317.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merage_text["transcript"] = merage_text["transcript"].apply(rmpunctuation)
/Users/lzw/packages/anaconda3/envs/ai/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 in labels with no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/lzw/packages/anaconda3/envs/ai/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 in labels with no true or predicted samples.

In [10]:
MELD_emo_test["predict_"] = MELD_emo_test["predict"].apply(parse_labels_MELD_emo)
MELD_emo_test["label_"] = MELD_emo_test["label"].apply(parse_labels_MELD_emo)
y_true = np.array(MELD_emo_test["label_"].tolist())
y_pred = np.array(MELD_emo_test["predict_"].tolist())
accuracy,micro_f1,macro_f1 = get_ACCandF1(y_true, y_pred)
print("MELD_emo:","acc", "mic-f1", "mac-f1")
print(accuracy,micro_f1,macro_f1)
results_write.extend([accuracy,macro_f1])

MELD_Sentiment_test["predict_"] = MELD_Sentiment_test["predict"].apply(extract_label_MELD_sen)
MELD_Sentiment_test["label_"] = MELD_Sentiment_test["label"].apply(extract_label_MELD_sen)
accuracy = accuracy_score(MELD_Sentiment_test['label_'], MELD_Sentiment_test['predict_'])
macro_f1 = f1_score(MELD_Sentiment_test['label_'], MELD_Sentiment_test['predict_'], average='macro')
print("MELD_sen:","acc", "mac-f1")
print(accuracy,macro_f1)
results_write.extend([accuracy,macro_f1])

CHSIMS_polarity_test["predict_"] = CHSIMS_polarity_test["predict"].apply(extract_label_CHSIMS_pol)
CHSIMS_polarity_test["label_"] = CHSIMS_polarity_test["label"].apply(extract_label_CHSIMS_pol)
accuracy = accuracy_score(CHSIMS_polarity_test['label_'], CHSIMS_polarity_test['predict_'])
macro_f1 = f1_score(CHSIMS_polarity_test['label_'], CHSIMS_polarity_test['predict_'], average='macro')
print("CHSIMS_sen:","acc", "mac-f1")
print(accuracy,macro_f1)
results_write.extend([accuracy,macro_f1])

CHSIMS_strength_test["predict_"] = CHSIMS_strength_test["predict"].apply(get_scores_CHSIMS_strength)
CHSIMS_strength_test["label_"] = CHSIMS_strength_test["label"].apply(get_scores_CHSIMS_strength)
correlation = CHSIMS_strength_test['predict_'].corr(CHSIMS_strength_test['label_'], method='pearson')
print("CHSIMS_strength")
print(correlation)
results_write.extend([correlation])

MELD_emo: acc mic-f1 mac-f1
0.47105899917335337 0.6296442687747036 0.469303636341926
MELD_sen: acc mac-f1
0.7090909090909091 0.6848621287144622
CHSIMS_sen: acc mac-f1
0.7221006564551422 0.6347286519723683
CHSIMS_strength
0.6747855302501787


In [11]:
df_results_write = pd.DataFrame([results_write])
df_results_write.to_csv("./results/results_video.csv", index=False, header=False, encoding="utf-8")